In [1]:
from fbprophet import Prophet

import pandas as pd
import numpy as np
from utils import get_data

from sklearn.preprocessing import StandardScaler
ssc = StandardScaler()

from preprocessing import select_df_columns, df_interpolate_and_dropna
from models import evaluate_model


In [2]:
%%time
df , df_raw = get_data('./data')
# df_temp = get_temperature_data("./data/temperature_brescia.json")
# df = df_temp.merge(df, how='outer', left_index=True, right_index=True)

Wall time: 29.5 s


C:\Users\matte\Documents\TAG\ProjectWork\utils.py:112: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_temp = df_temp.set_index("datetime")[["Temperatura"]].loc["2014":].resample('H').last()


In [3]:
del df_raw

In [4]:
%%time
df_prep = df.pipe(select_df_columns, ["ET_rete (potenza_termica_oraria)"])\
    .pipe(df_interpolate_and_dropna)

Wall time: 16 ms


In [5]:
del df

In [6]:
df_train = df_prep[["ET_rete (potenza_termica_oraria)"]][:"2019"]
df_test = df_prep[["ET_rete (potenza_termica_oraria)"]]["2020":]

In [7]:
df_train["ET_rete (potenza_termica_oraria)"] = ssc.fit_transform(df_train)
df_test["ET_rete (potenza_termica_oraria)"] = ssc.transform(df_test.values.reshape(-1,1))

In [8]:
df_train_prophet = df_train.reset_index()
df_train_prophet.columns = ['ds', 'y']
df_train_prophet

,ds,y
0,2014-01-01 00:00:00,-0.493770
1,2014-01-01 01:00:00,-0.486363
2,2014-01-01 02:00:00,-0.501178
3,2014-01-01 03:00:00,-0.506908
4,2014-01-01 04:00:00,-0.470570
...,...,...
52579,2019-12-31 19:00:00,1.824573
52580,2019-12-31 20:00:00,1.645528
52581,2019-12-31 21:00:00,1.413106
52582,2019-12-31 22:00:00,1.062167


In [9]:
%%time
model = Prophet(growth='linear',
                changepoints=None,
                n_changepoints=25,
                changepoint_range=0.8,
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=True,
                holidays=None,
                seasonality_mode='additive',
                seasonality_prior_scale=20.0,
                holidays_prior_scale=10.0,
                changepoint_prior_scale=0.05,
                mcmc_samples=0,
                interval_width=0.8,
                uncertainty_samples=1000,
                stan_backend=None)


Wall time: 1.54 s


In [10]:
%%time
model.fit(df_train_prophet)

Wall time: 1min 53s


In [11]:
future = model.make_future_dataframe(periods=48, include_history=False)
future

,ds
0,2020-01-01 23:00:00
1,2020-01-02 23:00:00
2,2020-01-03 23:00:00
3,2020-01-04 23:00:00
4,2020-01-05 23:00:00
5,2020-01-06 23:00:00
6,2020-01-07 23:00:00
7,2020-01-08 23:00:00
8,2020-01-09 23:00:00
9,2020-01-10 23:00:00


In [12]:
%%time
test_predictions = model.predict(future)

Wall time: 1.12 s


In [18]:
y_true = ssc.inverse_transform(df_test['ET_rete (potenza_termica_oraria)'].iloc[:48])

In [16]:
y_pred = ssc.inverse_transform(test_predictions['yhat'])

In [19]:
evaluate_model(y_true, y_pred)

mse = 8564.8214491659
r2_score = -1.0463032861283739
mape = 28.25771818423588
mae = 81.01376693209835
